## Tarefas a serem executadas

- Quando os dados estivem em branco, importar em branco;
- O telefone deve ser +55DDDNUMERO. Ex: (+5516981773421);
- O Valor deve ser formatado como dinheiro (real). Ex: 999,00;
- O valor_com_desconto deve ser calculado com o valor_total - desconto%;
- Datas no formato TIMESTAMP;
- Relacionar a tabela dependentes com usuários;

## Objetos e links

- Planilha do google sheets
    * https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/edit#gid=0
    * gid = 0 para planilha usuarios
    * gid = 822929440 para a planilha de dependentes


## Organização do documento e justificativas

1. A linguagem eleita foi o python em virtude de ferramentas simples e acessíveis para manipulação de planilhas e dados (pandas, regular expressions)
2. O pacote 'desafio' contém as funcções criada para executar as tarefas acima
3. Requerimentos: 
    - pandas (vantagem = performance e simplicidade) ; 
    - re (vantagem = manipulação de strings) ; 
    - numpy (para definir o tipo np.float64 e reduzir erros de arredondamento)
    - requests (para conectar ao google sheets)
4. Extração de planilha via api do google sheets
5. Após a importação dos pacotes, as célula seguintes correspondem às tarefas e suas respectivas respostas

In [1]:
## importação do pacote desafio
import __init__ as desafio

import pandas as pd
#
#
# import __init__ ao inves de import desafio pois o documento jupyter está no mesmo diretório do git


# link fornecido com a planilha formato xls (2 tabelas com dois gids diferentes)
url = 'https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/'

# cria uma lista de dataframes, uma para cada planilha
df  = desafio.create_dataframes_from_excel(url=url)

print('numero de planilhas:: %s' % (len(df)) )

numero de planilhas:: 2


## tarefa 1

> Quando os dados estivem em branco, importar em branco;

1. após a importação da planilha, as entradas não especificadas da tabela usuário aparecem como NaN.
2. a função sanitize_dataframes_nan procura e substitui todas as entradas NaN por espços em branco

In [3]:
# tarefa 1

desafio.sanitize_dataframes_nan(*df) # done!

#
# equivalente a sanitize_dataframes_nan(df[0],df[1])
#

print(df[0].head())


     id             nome                  email        telefone     valor  \
0  8056       Nia Nickle  noodles@sbcglobal.net    555919295629     45.45   
1  8094  Charisse Baltes          heine@aol.com                     45.44   
2  8087    Shaunna Myers         tezbo@yahoo.ca  (426) 296-8278      7.00   
3  8019      Del Mayorga      aukjan@icloud.com  (264) 385-2009     45.44   
4  8027   Kandis Marchan         lstein@mac.com  (209) 620-1483  10009.00   

  desconto  
0       35  
1       12  
2       45  
3        -  
4        8  


## tarefa 2

> O telefone deve ser +55DDDNUMERO. Ex: (+5516981773421)

1. a partir dos dados, observa-se que as configuraões dos números de telefones são: (DDD) 3NUMEROS-4NUMEROS ou 55DDDNUMERO $\rightarrow$ 10 dígitos
2. o modelo proposto foi de inserir o código de discagem direta internacional (DDI) do Brasil: **+55**
3. na função sanitize_phone_brazil assume-se que as entradas válidas possuem de 10 a 12 números
4. para selecionar todos os grupos de números é utilizada uma expressão regular excluir símbolos e em seguida faz a junção de todos os números
5. o conjunto de números que apresentar um tamanho menor que 10 ou maior que 12 são **inválidos** e a função retorna vazio
6. para os conjuntos com tamanho entre 10 e 12 a função insere +55 DDD 3NUMEROS-4NUMEROS
7. para melhorar a performance, utilizamos a função **apply** do pandas

OBS: a função apply equivale a função **map** do python, e executa a função desejada sobre cada entrada do vetor alvo. Em geral, as funções de map (e reduce) tendem a ser mais eficientes.

In [4]:
# tarefa 2

# atualização da coluna de número

df[0].telefone = df[0].telefone.apply(desafio.sanitize_phone_brazil)

print(df[0].head())

     id             nome                  email       telefone     valor  \
0  8056       Nia Nickle  noodles@sbcglobal.net  +555919295629     45.45   
1  8094  Charisse Baltes          heine@aol.com                    45.44   
2  8087    Shaunna Myers         tezbo@yahoo.ca  +554262968278      7.00   
3  8019      Del Mayorga      aukjan@icloud.com  +552643852009     45.44   
4  8027   Kandis Marchan         lstein@mac.com  +552096201483  10009.00   

  desconto  
0       35  
1       12  
2       45  
3        -  
4        8  


## tarefa 3

> O Valor deve ser formatado como dinheiro (real). Ex: 999,00;

1. a partir dos dados, observa-se que há mistura de pontos e virgulas. 
2. os dados podem ou não ter pontuação de milhar
3. a função sanitize_money usa expressões regulares ([0-9]+) para selecionar grupos de números
4. caso a expressão retorne **ao menos** dois grupos, o último grupo de inteiros representa centavos se possui no **máximo** dois dígitos
5. a coluna valor (ou columns[4]) contém os dados
6. para aumentar a performance, utiliza-se a função **apply** do pandas. O mesmo resultado poderia ser obtido por meio de um laço for


In [5]:
# tarefa 3

# sanitização do valor para real (padrão vírgula separa os centavos)

df[0].valor = df[0].valor.apply(desafio.sanitize_money)



print(df[0].head())

     id             nome                  email       telefone     valor  \
0  8056       Nia Nickle  noodles@sbcglobal.net  +555919295629     45,45   
1  8094  Charisse Baltes          heine@aol.com                    45,44   
2  8087    Shaunna Myers         tezbo@yahoo.ca  +554262968278      7,00   
3  8019      Del Mayorga      aukjan@icloud.com  +552643852009     45,44   
4  8027   Kandis Marchan         lstein@mac.com  +552096201483  10009,00   

  desconto  
0       35  
1       12  
2       45  
3        -  
4        8  


## tarefa 4

> O valor_com_desconto deve ser calculado com o valor_total - desconto%

1. após aplicar a função sanitize_money os valores estão no formato de reais e são do tipo string
2. para aplicar a função final_value remove-se a vírgula e é feita a conversão para inteiro (convert_tointeger)
3. alguns valores de desconto são inválidos, para corrigi-los é aplicada a função sanitize_discount, que transforma os valores invalídos em zero.
4. a operação realizada pela função consiste em multiplicar o valor do dinheiro por (1 - valor da porcetagem/100)
5. a função retorna o valor com desconto no formato de reais (convert_tostring)
6. criação de uma coluna com o valor com desconto



In [6]:
# tarefa 4

# sanitiza a coluna desconto
df[0].desconto = df[0].desconto.apply(desafio.sanitize_discount)

# criação da coluna valor_com_desconto
df[0]['valor_com_desconto']=[ desafio.final_value(df[0].valor[i],df[0].desconto[i]) for i in df[0].index ]

print(df[0].head())

     id             nome                  email       telefone     valor  \
0  8056       Nia Nickle  noodles@sbcglobal.net  +555919295629     45,45   
1  8094  Charisse Baltes          heine@aol.com                    45,44   
2  8087    Shaunna Myers         tezbo@yahoo.ca  +554262968278      7,00   
3  8019      Del Mayorga      aukjan@icloud.com  +552643852009     45,44   
4  8027   Kandis Marchan         lstein@mac.com  +552096201483  10009,00   

  desconto valor_com_desconto  
0       35              29,54  
1       12              39,99  
2       45               3,85  
3        0              45,44  
4        8            9208,28  


## tarefa 5

> Datas no formato TIMESTAMP;

1. os dados estão na planilha dependentes $\rightarrow$ df[1]
2. as datas são representadas na coluna data_hora ou columns[2]
2. aqui, a função intrínseca do pandas pd.to_datetime produz os resultados desejados ( motivo pelo qual não fizemos uma função sanitize_todatetime)
3. a função apply mapeia a função pd.to_datetime para cada entrada da coluna

In [8]:
#tarefa 5

#transforma string de data_hora para TIMESTAMP

df[1].data_hora = df[1].data_hora.apply( pd.to_datetime )

#
# pd.to_datetime pode gerar valores Not-a-Time (NaT) se as entradas estiverem em branco
#
# portanto, aplica-se sanitize novamente
desafio.sanitize_dataframes_nan(df[1])

print(df[1].head())

   id  user_id  dependente_id            data_hora
0   1     8056           8001  2019-03-07 09:46:33
1   2     8094           8017  2010-08-23 00:09:00
2   3     8087           8064                     
3   4     8019           8008  2019-03-13 09:46:33
4   5     8027           8081  2019-03-16 09:46:33


## tarefa 6

> Relacionar a tabela dependentes com usuários;

1. trata-se de uma instrução sql de merge de acordo com a chave do usuário
2. para o exemplo abaixo, utilizamos apenas um merge à esquerda (isto é, a tabela usuários é complementadas pelas informações da tabela dependente; se a opção 'inner' for utilizada, restringe-se a interseção dos bancos de acordo com as chaves utilizadas)
3. a função merge do pandas produz os resultados desejados mas convém encapsular a função para cópia ou eventual memoização (TODO sorry!)

In [10]:
#
# dataframe 0 recebe as entradas do dataframe 1, de acordo com as chaves que identificam os usuários
#
# copiamos o resultado da união dos dataframes para new_df

new_df = desafio.merge_by_key(df[0],df[1],key0='id',key1='user_id',how='left')

print(new_df.head())

   id_x             nome                  email       telefone  valor  \
0  8056       Nia Nickle  noodles@sbcglobal.net  +555919295629  45,45   
1  8094  Charisse Baltes          heine@aol.com                 45,44   
2  8087    Shaunna Myers         tezbo@yahoo.ca  +554262968278   7,00   
3  8019      Del Mayorga      aukjan@icloud.com  +552643852009  45,44   
4  8019      Del Mayorga      aukjan@icloud.com  +552643852009  45,44   

  desconto valor_com_desconto  id_y  user_id  dependente_id  \
0       35              29,54   1.0   8056.0         8001.0   
1       12              39,99   2.0   8094.0         8017.0   
2       45               3,85   3.0   8087.0         8064.0   
3        0              45,44   4.0   8019.0         8008.0   
4        0              45,44  17.0   8019.0         8033.0   

             data_hora  
0  2019-03-07 09:46:33  
1  2010-08-23 00:09:00  
2                       
3  2019-03-13 09:46:33  
4  2019-03-12 04:33:35  


***


# Debug

O arquivo __init__.py do pacote desafio contém as funções utilizada.

1. as funções possuem documentações simplificadas por meio de __doc__
2. se inicializado como um script, __init__ executa teste unitários para cada função do módulo
    - por exemplo, a função abaixo verifica se o dataframe foi criado corretamente a partir da url
    ```python
    def debug_excel():
        url = 'https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/'
        df = create_dataframes_from_excel(url=url)
        if df:
            return True
        return False
    ```
3. as funções de teste são:
    - debug_excel()
    - debug_csv()
    - debug_sanitize_nan()
    - debug_sanitize()
    - debug_sanitize_money()
    - debug_sanitize_phone_brazil()
    - debug_sanitize_discount()
    - debug_merge_by_key()



***

# Códigos

## Carregando dataframes:
```python
def download_data(url,option='xls',gid=None):
    """
    extrai planilha do google sheets utilizando a API google sheets.
    
    EXEMPLO: download_data('https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/',gid='0',option='csv')
    
    option: mime do arquivo. opções são xls e csv
    url   : url da planhilha. 
    gid   : id da planilha. Necessário para formato csv
    saida : dados da requisição GET
    """
    flavor_url = '/export?download'
    if (option == 'csv') & (gid is not None):
        flavor_url = '/export?format=csv&gid=%s' % gid
    base_url = 'https://docs.google.com/spreadsheets/d/'
    flag = url.find(base_url)
    if flag  <0:
        return None #'incorrect url. Try %s/KEY/' % base_url
    end_key = url[len(base_url):].find('/')
    if end_key < 0:
        return None #'incorrect url. Missing KEY. Try %s/KEY/' % base_url
    download_url = base_url+ url[len(base_url):(end_key+len(base_url))] + flavor_url
    #
    # TODO: usar memoização para evitar multiplos downloads
    #

    #
    # TODO: colocar clausulas *try* para erros de conexao
    #
    return rq.get(download_url)

def create_dataframes_from_excel(url=None,xls=None):
    """
    cria um conjunto de dataframes do pandas a partir de uma url valida para o google sheets 
    ou nome de arquivo excel
    
    parametros:
    url :: url que aponta para a planilha do google. EX: https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/
    xls :: caminho para o arquivo xls
    df  :: saida. lista de dataframes
    """
    if url:
        arquivo = download_data(url=url,option='xls') # TODO colocar uma clausula *if arquivo: ...*
        xls = 'arquivo_url.xls'
        with open(xls,'wb') as f:
            f.write(arquivo.content)
    # a string xls existe e tem valor definido
    if xls:
        sheets = pd.ExcelFile(xls).sheet_names # obtem todas as planilhas
        df = [pd.read_excel(xls, sheet_name = i) for i in sheets] # para cada planilha é criado um dataframe
        return df
    return None

def create_dataframes_from_csv(sheets=[],url=None,gids=None,sep=','):
    """
    Cria um conjunto de dataframes do pandas a partir de arquivos csv (sheets).
    Se os nomes dos arquivos csv não forem passados, assume-se que as planilhas
    serão baixadas por meio da url e das IDs das planilhas
    
    parametros:
    sheets :: lista com os caminhos dos arquivos csv
    url    :: url da planilha do google
    gids   :: lista com ID de cada planilha (não é a key da planilha)
    sep    :: separador dos arquivos csv
    df     :: saída. lista de dataframes
    """
    df=[]
    if sheets:
        #
        # TODO : chamar sys para verificar se arquivo csv existe
        #
        for sheet in sheets:
            df.append( pd.read_csv( sheet ,sep = sep ) )
        return df

    if (url is not None) & (gids is not None):
        sheets=[]
        for x in gids:
            arquivo = download_data(url=url,gid=x,option='csv')
            sheets.append( arquivo.text )
    from io import StringIO
    for sheet in sheets:
        df.append( pd.read_csv( StringIO(sheet) ,sep = sep ) )
    return df

```
## Sanitização:

```python
def sanitize_dataframes_nan(*dataframe):
    """
    substitui os locais sem preenchimento (NaN) por espaço vazio
    
    parametro:
    dataframe :: conjunto de dataframes
    """
    for data in dataframe:
        data.fillna('', inplace=True)
    return

def sanitize_phone_brazil(phonenumber):
    """
    transforma o número de telefone para o formato +55DDDNUMERO
    
    parametros:
    phonenumber :: numero do telefone a ser formatado
    numero      :: saida
    """
    number_size = 10      # configuração dos número:      3DDD 3NUMEROS-4NUMEROS.
    number_size_max = 12  # configuração dos número: 2DDI 3DDD 3NUMEROS-4NUMEROS
    res = re.findall('([0-9]+)',str(phonenumber))
    number=''.join(res)
    if (len(number) < number_size) or (len(number) > number_size_max):
        return ''
    return '+55'+number[-number_size:]


def sanitize_money(money):
    """
    transforma o dinheiro para o formato NUMEROS_REAIS,NUMEROS_CENTAVOS
    
    parametros:
    money  :: valor do dinheiro a ser formatado
    """
    moneys = re.findall( '([0-9]+)',str(money))
    cents  = '00'
    if (len(moneys) > 0):
        if (len(moneys)>1) & (len(moneys[-1]) < 3) : # centavos tem no maximo 2 digitos
            cents = ''.join(moneys[-1])
            cents = cents.ljust(2,'0')
            moneys.pop(-1)
        return ''.join(moneys)+','+cents
    return ''


def sanitize_discount(desconto):
    """
    retira os caracteres inválidos dos valores de desconto, substituindo-os por 0
    
    parametro:
    desconto :: valor do desconto
    """
    return ''.join(re.findall('[0-9]+', str(desconto))) or '0'


```
## Operações com números (int e float) e strings:
```python
def convert_tointeger(var_input):
    """
    converte as entradas do dinheiro para inteiro
    
    parametro:
    var_input :: valor do dinheiro
    """
    return int(sanitize_money(var_input).replace(',',''))

def convert_tostring(var_input):
    """
    converte a entrada para string
    
    parametro:
    var_input :: valor do dinheiro
    
    """
    return "%.2f" % var_input


def final_value(value,discount='0'):
    """
    calcula o valor com o desconto
    
    parametros:
    value    :: valor do dinheiro
    discount :: valor do desconto
    saida    :: valor com desconto
    """
    money = np.float64(convert_tointeger(value))/100.0
    delta = np.float64(discount)/100.0
    return sanitize_money(convert_tostring(money*(1e0 - delta)))
```

## Associação
```python
def merge_by_key(df0,df1,key0='id',key1='user_id',how='left'):
    """
    retorna a junção de duas tabelas de acordo com campos em comum key0 e key1
    
    df0  :: dataframe alvo.
    df1  :: dataframe auxiliar
    key0 :: string. chave de df0.
    key1 :: string. chave de df1.
    how  :: string. maneira como o merge é feito. pode ser left, right, inner
    saida:: df0 é atualizada com os campos correspondentes de df1
    """
    #return df0.merge(df1,left_on=key0,right_on=key1,how=how)
    return pd.merge(df0,df1,left_on=key0,right_on=key1,how=how)

```

## debug


```python
    def debug_excel():
        url = 'https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/'
        df = create_dataframes_from_excel(url=url)
        if df:
            return True
        return False

    def debug_csv():
        url = 'https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/'
        df = create_dataframes_from_csv(url=url,gids=['0','822929440'])
        if df:
            return True
        return False
    
    def debug_sanitize_nan():
        url = 'https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/'
        df = create_dataframes_from_excel(url=url)
        sanitize_dataframes_nan(*df)
        #
        # TODO:
        #      df.isna() retorna True mesmo quando não há valores NaN
        #      workaround: define função (poderia ser lambda tbm)
        #      que usa .isna() em cada coluna do dataframe
        #
        def helper(data):
            return any( [ any(data[x].isna()) for x in data.columns] )
        return not any( [helper(data) for data in df] )
    
    def debug_sanitize(tentativa,f):
        # tentativa é um dicionário que contém as chaves num formato, e o dado correspondente no formato desejado
        return not any( [ not (f(x) == tentativa[x]) for x in tentativa] )
    
    def debug_sanitize_money():
        # tentativa é um dicionário que contém as chaves num formato, e o dado correspondente no formato desejado
        tentativa={'1.1':'1,10', '7':'7,00' , '1.230':'1230,00' , '1.230.20': '1230,20' }
        return debug_sanitize(tentativa,sanitize_money)
    def debug_sanitize_phone_brazil():
        # tentativa é um dicionário que contém as chaves num formato, e o dado correspondente no formato desejado
        tentativa={'+559991234567':'+559991234567','(999)1234567':'+559991234567','999123456':'','+5999123-4567':'+559991234567', '9'*13:''}
        return debug_sanitize(tentativa,sanitize_phone_brazil)
    def debug_sanitize_discount():
        # tentativa é um dicionário que contém as chaves num formato, e o dado correspondente no formato desejado
        tentativa={'3':'3','30':'30','-':'0'}
        return debug_sanitize(tentativa,sanitize_discount)
    def debug_merge_by_key():
        url = 'https://docs.google.com/spreadsheets/d/1N6JFMIQR71HF5u5zkWthqbgpA8WYz_0ufDGadeJnhlo/'
        df = create_dataframes_from_csv(url=url,gids=['0','822929440'])
        DF = merge_by_key(df[0],df[1],how='inner')
        return (len(DF) == len(df[1]))

```